#### Инструкция
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* video.ittensive.com/machine-learning/prudential/train.csv.gz
* video.ittensive.com/machine-learning/prudential/test.csv.gz
* video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

#### Задача
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

### Подключение библиотек и функции оптимизации памяти

In [1]:
import numpy as np 
import pandas as pd

# библиотеки для ансамбля 
from catboost import Pool, CatBoostClassifier  # CatBoost
from sklearn.ensemble import GradientBoostingClassifier  # Градиентный бустинг
from xgboost import XGBClassifier  # XGBoost
import lightgbm as lgb  # LightGBM

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn import preprocessing


def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

Загрузка данных, приведение их к числовым, заполнение пропусков, нормализация данных и оптимизация памяти.

### Загрузка данных

In [2]:
# data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
data = pd.read_csv("train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных
Создадим отдельную функцию, для предобработки данных. Её нужно будет применить как к обч. так и к тест. данным. 

In [3]:
def data_preprocess (df):
    df["Product_Info_2_1"] = df["Product_Info_2"].str.slice(0, 1)
    df["Product_Info_2_2"] = pd.to_numeric(df["Product_Info_2"].str.slice(1, 2))
    df.drop(labels=["Product_Info_2"], axis=1, inplace=True)
    for l in df["Product_Info_2_1"].unique():
        df["Product_Info_2_1" + l] = df["Product_Info_2_1"].isin([l]).astype("int8")
    df.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
    df.fillna(value=-1, inplace=True)
    data["Response"] = data["Response"] - 1
    return df

Проведем предобработку тренировочных данных.   

In [4]:
data = data_preprocess(data)

### Набор столбцов для расчета
Создадим список столбцов для расчета в модели (вкл. исправление опечатки в InsuredInfo)

In [5]:
columns_groups = ["Insurance_History", "InsuredInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keywor

### Нормализация данных 

In [6]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти
Воспользуемся функцией оптимизации памяти для сформированных данных.

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 42.87 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 126 entries, 0 to Response
dtypes: float16(125), int8(1)
memory usage: 14.2 MB
None


### Задача
Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

### Построение базовых моделей
Сформируем параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Использовать будем лучшие гиперпараметры, который были подобраны ранее. 

In [8]:
x = pd.DataFrame(data_transformed, columns=columns_transformed)

CatBoost. Оптимальные парамеры которые нашли ранее - 'depth': 7, 'l2_leaf_reg': 1, 'learning_rate': 0.5700000000000001

In [9]:
model_cat_boost = CatBoostClassifier(iterations=100,
        learning_rate=0.5700000000000001,
        depth=7,
        l2_leaf_reg=1,
        random_seed=17, loss_function="MultiClass",
        bootstrap_type="MVS", custom_metric="WKappa")
train_dataset = Pool(data=x, label=data_transformed["Response"])
model_cat_boost.fit(train_dataset)

0:	learn: 1.5279394	total: 257ms	remaining: 25.5s
1:	learn: 1.4322561	total: 376ms	remaining: 18.4s
2:	learn: 1.3338654	total: 511ms	remaining: 16.5s
3:	learn: 1.2977580	total: 632ms	remaining: 15.2s
4:	learn: 1.2739421	total: 744ms	remaining: 14.1s
5:	learn: 1.2594011	total: 855ms	remaining: 13.4s
6:	learn: 1.2479667	total: 965ms	remaining: 12.8s
7:	learn: 1.2403408	total: 1.07s	remaining: 12.4s
8:	learn: 1.2278383	total: 1.21s	remaining: 12.3s
9:	learn: 1.2176685	total: 1.35s	remaining: 12.1s
10:	learn: 1.2127665	total: 1.46s	remaining: 11.8s
11:	learn: 1.2079266	total: 1.58s	remaining: 11.6s
12:	learn: 1.1996008	total: 1.69s	remaining: 11.3s
13:	learn: 1.1917714	total: 1.8s	remaining: 11.1s
14:	learn: 1.1835361	total: 1.91s	remaining: 10.8s
15:	learn: 1.1794947	total: 2.04s	remaining: 10.7s
16:	learn: 1.1758466	total: 2.18s	remaining: 10.6s
17:	learn: 1.1729276	total: 2.29s	remaining: 10.4s
18:	learn: 1.1601366	total: 2.4s	remaining: 10.2s
19:	learn: 1.1567989	total: 2.51s	remaining

Градиентный бустинг. Оптимальные парамеры которые нашли ранее - 'max_depth': 15, 'max_features': 26, 'min_samples_leaf': 20, 'n_estimators': 75

In [10]:
# параметры от случайного леса: max_depth=17, max_features=27, min_samples_leaf=20, n_estimators=76.

model_gb = GradientBoostingClassifier(random_state=17,
        min_samples_leaf=20,
        max_features=26,
        max_depth=15,
        n_estimators=75)
model_gb.fit(x, data_transformed["Response"])

GradientBoostingClassifier(max_depth=15, max_features=26, min_samples_leaf=20,
                           n_estimators=75, random_state=17)

XGBoost. Параметры от случайного леса: max_depth=17, max_features=27, min_samples_leaf=20, n_estimators=76

In [11]:
model_xgb = XGBClassifier(max_depth=17, 
            max_features=27,
            n_estimators=76, 
            min_samples_leaf=20)
model_xgb.fit(x, data_transformed["Response"])

[11:36:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf" } are not used.



XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=17, max_features=27, max_leaves=0,
              min_child_weight=1, min_samples_leaf=20, missing=nan,
              monotone_constraints='()', n_estimators=76, n_jobs=0,
              num_parallel_tree=1, ...)

LightGBM, используем multiclass. Оптимальные парамеры которые нашли ранее - 'max_depth': 17, 'min_child_samples': 18, 'num_leaves': 34

In [12]:
model_lgb = lgb.LGBMRegressor(random_state=17, max_depth=18,
            min_child_samples=18, num_leaves=75, n_estimators=1000,
            objective="multiclass", num_class=8)
model_lgb.fit(x, data_transformed["Response"])

LGBMRegressor(max_depth=18, min_child_samples=18, n_estimators=1000,
              num_class=8, num_leaves=75, objective='multiclass',
              random_state=17)

### Расчет предказаний
Кроме непосредственно значений дополнительно посчитаем вероятности совпадения с тем или иным классом. Это потребуется для ансамбля голосования. 

In [13]:
x_test = pd.DataFrame(data_transformed, columns=columns_transformed)

In [14]:
data_test_catb_proba = pd.DataFrame(model_cat_boost.predict_proba(x_test))
data_test_gb_proba = pd.DataFrame(model_gb.predict_proba(x_test))
data_test_xgb_proba = pd.DataFrame(model_xgb.predict_proba(x_test))
data_test_lgb_proba = pd.DataFrame(model_lgb.predict(x_test))

### Оценка ансамбля
Рассчитаем оценку взвешенного предсказания 4 моделей

In [15]:
def vote_class (x):
    a = np.argmax(x.values)
    return a

In [16]:
data_test_proba = data_test_catb_proba.copy()
for i in range(0, 8):
    data_test_proba[i] = 1*data_test_proba[i]
    data_test_proba[i] = data_test_proba[i] + 1*data_test_gb_proba[i]
    data_test_proba[i] = data_test_proba[i] + 1*data_test_xgb_proba[i]
    data_test_proba[i] = data_test_proba[i] + 1*data_test_lgb_proba[i]
data_test_proba["target"] = data_test_proba.apply(vote_class, axis=1)

print ("Ансамбль (CatBoost, GBoost, XGBoost и LightGBM) классификации:",
      round(cohen_kappa_score(data_test_proba["target"],
                data_transformed["Response"], weights="quadratic"), 3))


Ансамбль (CatBoost, GBoost, XGBoost и LightGBM) классификации: 0.982


In [17]:
print(data_test_proba["target"].head(15))
print(data_transformed["Response"].head(15))

0     7
1     3
2     7
3     7
4     7
5     7
6     7
7     0
8     7
9     0
10    5
11    1
12    6
13    2
14    7
Name: target, dtype: int64
0     7
1     3
2     7
3     7
4     7
5     7
6     7
7     0
8     7
9     0
10    5
11    1
12    6
13    2
14    7
Name: Response, dtype: int8


### Загрузка данных для расчетов
Применим построенный ансамбль моделей для расчета актуальных данных (актуал. значения классификации).

Будем использовать ранее написанные функции для нормализация и оптимизации данных.

In [18]:
# data_test = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/test.csv.gz")
data_test = pd.read_csv("test.csv.gz")
data_test = data_preprocess(data_test)
data_test = reduce_mem_usage(data_test)
data_test_transformed = pd.DataFrame(scaler.transform(pd.DataFrame(data_test,
                                            columns=columns)))
print (data_test_transformed.info())

Потребление памяти меньше на 16.34 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 125 entries, 0 to 124
dtypes: float32(125)
memory usage: 9.4 MB
None


In [19]:
print(data_test_transformed.head())

        0         1         2         3         4         5         6    \
0  0.519789  1.002921  1.045952  0.022140  0.611857 -0.169414 -1.159587   
1  0.215425  0.266273  1.122714  0.126020 -1.634368 -0.169414  0.862391   
2  0.308653  0.022915  0.894903  0.405695  0.611857 -0.169414 -1.159587   
3 -0.278139 -0.707156  0.592804  0.143999 -1.634368 -0.169414  0.862391   
4 -0.513953 -0.463799 -0.542540 -0.333447  0.611857 -0.169414 -1.159587   

        7         8         9    ...       115       116       117       118  \
0  1.101046 -1.156735  1.130555  ...  0.559558 -0.083689  0.441621 -0.149284   
1 -1.013721  0.864261 -0.928723  ... -0.892015 -0.083689  0.441621 -0.149284   
2  1.101046 -1.156735  1.130555  ... -0.652247 -0.083689  0.441621 -0.149284   
3 -1.013721  0.862242  0.100916  ... -0.627190 -0.083689 -2.264385 -0.149284   
4  1.101046 -1.156735  1.130555  ... -0.892015 -0.083689  0.441621 -0.149284   

        119       120       121       122       123       124  
0 -0

### Расчет предсказаний
Произведем предсказания класса по ранее построенному ансамблю 

In [20]:
data_test_catb = pd.DataFrame(model_cat_boost.predict_proba(Pool(data=data_test_transformed)))
data_test_gb = pd.DataFrame(model_gb.predict_proba(data_test_transformed))
data_test_xgb = pd.DataFrame(model_xgb.predict_proba(data_test_transformed))
data_test_lgb = pd.DataFrame(model_lgb.predict(data_test_transformed))

In [21]:
data_test_1 = data_test_catb.copy()
for i in range(0, 8):
    data_test_1[i] = data_test_1[i]
    data_test_1[i] = data_test_1[i] + 1*data_test_gb[i]
    data_test_1[i] = data_test_1[i] + 1*data_test_xgb[i]
    data_test_1[i] = data_test_1[i] + 1*data_test_lgb[i]
data_test_transformed["Response"] = (data_test_1.apply(vote_class, axis=1) + 1).astype("int8") 

In [26]:
print(data_test_transformed["Response"].head())
print(data_test_transformed["Response"])

0    1
1    8
2    6
3    8
4    8
Name: Response, dtype: int8
0        1
1        8
2        6
3        8
4        8
        ..
19760    8
19761    8
19762    6
19763    2
19764    4
Name: Response, Length: 19765, dtype: int8


### Формирование результатов
Загрузим пример данных для отправки и заменим в нем столбец Response на рассчитанный ранее.

In [23]:
# submission = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz")
submission = pd.read_csv("sample_submission.csv.gz")

print (submission.head())

   Id  Response
0   1         8
1   3         8
2   4         8
3   9         8
4  12         8


In [24]:
submission["Response"] = data_test_transformed["Response"].astype("int8")
print (submission.head())

   Id  Response
0   1         1
1   3         8
2   4         6
3   9         8
4  12         8


### Выгрузка результатов

In [25]:
submission.to_csv("submission.csv", index=False)